<a href="https://colab.research.google.com/github/fahernkhan/Natural-Language-Processing-NLP-/blob/main/Multiclass_Text_Classification_menggunakan_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

menentukan genre sebuah film berdasarkan sinopsisnya
dataset : https://www.kaggle.com/datasets/antoniuscs/imdb-synopsis-indonesian-movies

In [5]:
import pandas as pd
from google.colab import files
files.upload()
df = pd.read_csv('imdb_indonesian_movies_2.csv')
df = df.drop(columns=['judul_film'])

Saving imdb_indonesian_movies_2.csv to imdb_indonesian_movies_2 (1).csv


Panggil fungsi head() pada dataframe untuk menampilkan 5 sampel teratas pada dataset.

In [7]:
df.head()

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


Karena label kita berupa data kategorikal, maka kita perlu melakukan proses one-hot-encoding. Jalankan kode di bawah untuk melakukan one-hot-encoding dan membuat dataframe baru.

In [8]:
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='genre')
df_baru

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


Agar dapat diproses oleh model, kita perlu mengubah nilai-nilai dari dataframe ke dalam tipe data numpy array menggunakan atribut values.

In [9]:
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

bagi data untuk training dan data untuk testing.

In [10]:
from sklearn.model_selection import train_test_split
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

Kemudian kita ubah setiap kata pada dataset kita ke dalam bilangan numerik dengan fungsi Tokenizer. Setelah tokenisasi selesai, kita perlu membuat mengonversi setiap sampel menjadi sequence.

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(sinopsis_latih) 
tokenizer.fit_on_texts(sinopsis_test)
 
sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)
 
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

Untuk arsitektur model kita menggunakan layer Embedding dengan dimensi embedding sebesar 16, serta dimensi dari input sebesar nilai num_words pada objek tokenizer. Jangan lupa panggil fungsi compile dan tentukan optimizer serta loss function yang akan dipakai oleh model.

In [12]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

Terakhir kita dapat mulai melatih model kita dengan memanggil fungsi fit().

In [13]:
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs, 
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/30
26/26 - 39s - loss: 1.6101 - accuracy: 0.1940 - val_loss: 1.6104 - val_accuracy: 0.1741 - 39s/epoch - 2s/step
Epoch 2/30
26/26 - 30s - loss: 1.6075 - accuracy: 0.2114 - val_loss: 1.6102 - val_accuracy: 0.1741 - 30s/epoch - 1s/step
Epoch 3/30
26/26 - 33s - loss: 1.5774 - accuracy: 0.3060 - val_loss: 1.5972 - val_accuracy: 0.2139 - 33s/epoch - 1s/step
Epoch 4/30
26/26 - 30s - loss: 1.4256 - accuracy: 0.4042 - val_loss: 1.5861 - val_accuracy: 0.2736 - 30s/epoch - 1s/step
Epoch 5/30
26/26 - 30s - loss: 0.9898 - accuracy: 0.5473 - val_loss: 2.0371 - val_accuracy: 0.2736 - 30s/epoch - 1s/step
Epoch 6/30
26/26 - 32s - loss: 0.6852 - accuracy: 0.6517 - val_loss: 2.2615 - val_accuracy: 0.2687 - 32s/epoch - 1s/step
Epoch 7/30
26/26 - 30s - loss: 0.5668 - accuracy: 0.7413 - val_loss: 2.5279 - val_accuracy: 0.2836 - 30s/epoch - 1s/step
Epoch 8/30
26/26 - 30s - loss: 0.3823 - accuracy: 0.8172 - val_loss: 2.9994 - val_accuracy: 0.2786 - 30s/epoch - 1s/step
Epoch 9/30
26/26 - 32s - loss: 0